# Web Scraping from Scratch with Selenium & AWS

## Objective
1. Scrape top  10 trending videos on Youtube using Selenium
2. Set up a recurring job on AWS Lambda to scrape every 30 minute
3. Send the result as a CSV attachment over email (or to a spreadsheet)

### Step 1 - Create a GitHub repository
* Create a repository at https://github.com/new
* Add README, gitignore (Python) and license 
* (Optional) Clone the repository locally
* References:
    * Introduction to GitHub: https://lab.github.com/githubtraining... 
    * Git & GitHub tutorial:   

 • Git and GitHub fo...   


### Step 2 - Launch the repository on Replit
* Connect Replit with your GitHub account
* Launch the repository as a Replit project
* Set up the language and run command
* Create and execute a Python script
* Attempt to scrape the page using requests & Beautiful Soup
* References:
    * Introduction to Replit: https://docs.replit.com/tutorials/01-... 
    * Replit + GitHub: https://docs.replit.com/tutorials/06-... 
    * YouTube trending feed: https://www.youtube.com/feed/trending 
    * Beautiful soup tutorial: https://blog.jovian.ai/web-scraping-u... 


### Step 3 - Extract information using Selenium
* Install selenium and create a browser driver
* Load the page and extract information
* Create a CSV of results using Pandas
* References:
    * Selenium tutorial: https://www.browserstack.com/guide/py...
    * Pandas tutorial: https://jovian.ai/learn/data-analysis...


### Step 4 - Set up a recurring job on AWS Lambda
* Create an AWS Lambda Python function
* Deploy a sample script and observe the output
* Add layers for Selenium and Chromium
* Set up recurring job using AWS CloudWatch
* References:
    * Python on AWS Lambda tutorial: https://stackify.com/aws-lambda-with-... 
    * Chromium & Selenium on AWS Lambda: https://dev.to/awscommunity-asean/cre...
    * Recurring AWS Lambda functions: https://docs.aws.amazon.com/lambda/la... 

### Step 5 - Send results over email using SMTP
* Create email client using smtplib
* Set up SSL, TLS and authenticate with password
* Send a sample email with just text
* Send an email with text and attachment
* References:
    * Sending Email with Python: https://stackabuse.com/how-to-send-em...
    * Send email using Python: https://www.geeksforgeeks.org/send-ma...
    * Environment variables on Replit: https://docs.replit.com/programming-i...
    * https://docs.aws.amazon.com/lambda/la... 
    * Update Google sheets using Python: https://www.analyticsvidhya.com/blog/...

In [4]:
## install beautiful soup and requests library
## Selenium - is python API which interact with webrowser, you'd still need # a web driver to connect
pip install bs4 requests selenium

SyntaxError: invalid syntax (1262808197.py, line 3)

In [6]:
pip install selenium

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 1.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.9/384.9 KB 1.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 KB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 KB 1.8 MB/s eta 0:00:00
You should consider upgrading via the '/Users/mac/.pyenv/versions/3.8.15/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import requests
from bs4 import BeautifulSoup

YOUTUBE_TRENDING_URL = 'https://www.youtube.com/feed/trending'

# request.get post javascript down but does not execute JavaScript - all those data getting dynamically is not
# happening here as there is no video on the intial page
response = requests.get(YOUTUBE_TRENDING_URL)

# status code of "200" means successful , "404" means unsuccessful
print('Status Code', response.status_code)

# w means "write to the file"
# with open('trending.html', 'w') as f:
#     f.write(response.text)

doc = BeautifulSoup(response.text, 'html.parser')

print('Page title', doc.title.text)

# Find all the video divs
video_divs = doc.find_all('div',
                          class_ = 'ytd-video-renderer')

print(f'Found {len(video_divs)} videos')

Status Code 200
Page title มาแรง - YouTube
Found 0 videos


In [6]:
response.headers

{'Content-Type': 'text/html; charset=utf-8', 'X-Content-Type-Options': 'nosniff', 'Cache-Control': 'no-cache, no-store, max-age=0, must-revalidate', 'Pragma': 'no-cache', 'Expires': 'Mon, 01 Jan 1990 00:00:00 GMT', 'Date': 'Thu, 02 Mar 2023 06:07:19 GMT', 'X-Frame-Options': 'SAMEORIGIN', 'Strict-Transport-Security': 'max-age=31536000', 'Report-To': '{"group":"youtube_main","max_age":2592000,"endpoints":[{"url":"https://csp.withgoogle.com/csp/report-to/youtube_main"}]}', 'Permissions-Policy': 'ch-ua-arch=*, ch-ua-bitness=*, ch-ua-full-version=*, ch-ua-full-version-list=*, ch-ua-model=*, ch-ua-wow64=*, ch-ua-platform=*, ch-ua-platform-version=*', 'Cross-Origin-Opener-Policy-Report-Only': 'same-origin-allow-popups; report-to="youtube_main"', 'P3P': 'CP="This is not a P3P policy! See http://support.google.com/accounts/answer/151657?hl=th for more info."', 'Content-Encoding': 'gzip', 'Server': 'ESF', 'X-XSS-Protection': '0', 'Set-Cookie': 'GPS=1; Domain=.youtube.com; Expires=Thu, 02-Mar-202

In [3]:
chromedriver --version

NameError: name 'chromedriver' is not defined

In [61]:
# let's start fresh after installing Selenium for simplicity reasons
""" Sometimes the code will not scrape all the videos if the page is not loaded completely. 
  Import the time module & use the time.sleep(5) command to load the page completely and then find the elements. """
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
# from selenium.webdriver.chrome.service import Service

YOUTUBE_TRENDING_URL = 'https://www.youtube.com/feed/trending'

def get_driver():
    # get chrome driver - u can think of the driver as browser
    options = Options()
    # options.headless = True
    options.add_argument("--headless=new")
    # ser = Service(r"./chromedriver.exe")
    driver = webdriver.Chrome("./chromedriver", options=options)
    return driver

def get_videos(driver):
    
    # With the same metaphor, you can insert URL to the browser
    driver.get(YOUTUBE_TRENDING_URL)

    # Getting the video divs
    video_div_tag = 'ytd-video-renderer'
    time.sleep(5) #
    video_div = driver.find_elements(By.TAG_NAME, video_div_tag) # Source: https://stackoverflow.com/questions/69875125/find-element-by-commands-are-deprecated-in-selenium
    # video_div = driver.find_elements_by_class_name(video_div_class)
    return video_div

if __name__ == "__main__":

    print('Creating Driver')
    driver = get_driver()

    # With the same metaphor, you can insert URL to the browser
    print('Fetching trending video')
    videos = get_videos(driver)
    
    print(f'Found {len(videos)} videos')

    print('Parsing the the first video')
    # title, url, thumbnail_url channel, views, uploaded, description

    video = videos[0]
    title_tag = video.find_element(By.ID, 'video-title')
    title = title_tag.text
    url = title_tag.get_attribute('href')
    print('Title', title)
    print('URL', url)



Creating Driver


/var/folders/7h/vk9jt7z14ddd1__kc33kl45w0000gn/T/ipykernel_64260/1139820817.py:18: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("./chromedriver", options=options)


Fetching trending video
Found 97 videos
Parsing the the first video
Title ILLSLICK - My Dad [Official Music Video]
URL https://www.youtube.com/watch?v=T3Fv4IoatxE
